In [1]:
ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data"

In [2]:
import os
import json
import cv2
from tqdm import tqdm
import shutil
from multiprocessing import Pool, cpu_count

# 변환 비율
scale_x = 1280 / 3840
scale_y = 720 / 2160

ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data"
OUT = ROOT + "_resized"
os.makedirs(OUT, exist_ok=True)

# ----(1) 디렉토리를 먼저 전부 생성해두기 ----
def build_all_dirs():
    for root, dirs, files in os.walk(ROOT):
        rel = os.path.relpath(root, ROOT)
        out_dir = os.path.join(OUT, rel)
        os.makedirs(out_dir, exist_ok=True)

def resize_image_fast(img_path, out_path):
    img = cv2.imread(img_path, cv2.IMREAD_REDUCED_COLOR_4)
    if img is None:
        return
    resized = cv2.resize(img, (1280, 720), interpolation=cv2.INTER_AREA)
    cv2.imwrite(out_path, resized)

def transform_bbox(bbox_str):
    x1, y1, x2, y2 = map(float, bbox_str.split(","))
    x1 *= scale_x; y1 *= scale_y
    x2 *= scale_x; y2 *= scale_y
    return f"{x1},{y1},{x2},{y2}"

def convert_json(json_path, out_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if "bboxdata" in data:
        data["bboxdata"]["bbox_location"] = transform_bbox(data["bboxdata"]["bbox_location"])

    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ----(2) 파일만 처리 (디렉토리 생성 없음) ----
def process_file(src_path):
    ext = src_path.lower().split(".")[-1]
    rel_path = os.path.relpath(src_path, ROOT)
    dst_path = os.path.join(OUT, rel_path)

    if ext in ["jpg", "jpeg", "png"]:
        resize_image_fast(src_path, dst_path)

    elif ext == "json":
        convert_json(src_path, dst_path)


def process_all():
    print("📁 디렉토리 구조 생성 중...")
    build_all_dirs()

    all_files = []
    for root, dirs, files in os.walk(ROOT):
        for file in files:
            all_files.append(os.path.join(root, file))

    print(f"총 파일 수: {len(all_files)}")

    with Pool(cpu_count()) as pool:
        list(tqdm(pool.imap_unordered(process_file, all_files),
                   total=len(all_files),
                   desc="Processing"))

process_all()
print("🚀 변환 완료 (멀티프로세싱 + 빠른 로드 + 안전한 디렉토리 생성)")

📁 디렉토리 구조 생성 중...
총 파일 수: 113280


Processing: 100%|██████████| 113280/113280 [4:40:50<00:00,  6.72it/s]


🚀 변환 완료 (멀티프로세싱 + 빠른 로드 + 안전한 디렉토리 생성)
